# My portfolios

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import robin_stocks as r
import etfs as et

/Users/akuepper/anaconda/envs/etfs_dev/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/akuepper/anaconda/envs/etfs_dev/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/akuepper/anaconda/envs/etfs_dev/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
from etfs.portfolio.io import import_portfolio, import_portfolio_robinhood, import_portfolio_vanguard, \
                              parse_portfolio, parse_portfolio_vanguard
from etfs.stats.basics import resample_df, runrate_column, shift_column, \
                              standard_deviation_column, difference, rsq, beta, alpha
from etfs.utils.helpers import todays_date
from etfs.visualization.trends import plot_composition
from etfs.visualization.analysis import plot_cross_correlation_matrix

/Users/akuepper/anaconda/envs/etfs_dev/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## RobinHood

In [4]:
# access password file
f = open("../../../.etfs.txt","r")
lines = f.readlines()
username = lines[0].strip().split('\n', 0)
password = lines[1].strip().split('\n', 0)
f.close()

In [5]:
access_token = r.login(username, password)

Please Type In The MFA Code: 648940


In [8]:
rh = import_portfolio_robinhood(access_token=access_token, 
                                username=username, 
                                password=password, 
                                name="Robinhood", 
                                free_stock=True)

Found Additional pages.
Loading page 2 ...
Loading page 3 ...
Loading page 4 ...
Loading page 5 ...
Loading page 6 ...
Loading page 7 ...
Loading page 8 ...
Loading page 9 ...
Loading page 10 ...
Loading page 11 ...
Loading page 12 ...
Loading page 13 ...
Parsing orders ...
Parsing bank transfers ...
Parsing dividends ...
Adding promotional stock ...
depositing 550.00 USD (new balance: 550.00 USD)
Checking ../data/MSFT.csv
Saving ../data/MSFT.csv
Checking ../data/MSFT.csv
Saving ../data/MSFT.csv
buying 1.00 MSFT (new balance: 455.88 USD)
buying 1.00 MSFT (new balance: 361.52 USD)
Checking ../data/AAPL.csv
Saving ../data/AAPL.csv
Checking ../data/AAPL.csv
Saving ../data/AAPL.csv
buying 1.00 AAPL (new balance: 193.84 USD)
Checking ../data/BABA.csv
Saving ../data/BABA.csv
Checking ../data/BABA.csv
Saving ../data/BABA.csv
buying 1.00 BABA (new balance: 0.50 USD)
Checking ../data/CHK.csv
Saving ../data/CHK.csv
Checking ../data/CHK.csv
Saving ../data/CHK.csv
buying 1.00 CHK (new balance: 0.5

Checking ../data/NTDOY.csv
Saving ../data/NTDOY.csv
Checking ../data/NTDOY.csv
Saving ../data/NTDOY.csv
buying 1.00 NTDOY (new balance: 67.93 USD)
buying 5.00 NIO (new balance: 35.29 USD)
Checking ../data/CVS.csv
Saving ../data/CVS.csv
Checking ../data/CVS.csv
Saving ../data/CVS.csv
buying 1.00 CVS (new balance: -30.73 USD)
selling 2.00 IQ (new balance: 5.55 USD)
buying 4.00 NIO (new balance: -20.71 USD)
buying 9.00 NTDOY (new balance: -333.42 USD)
selling 1.00 NFLX (new balance: 3.58 USD)
buying 2.00 ZTO (new balance: -30.88 USD)
selling 3.00 SONO (new balance: 2.90 USD)
depositing 1000.00 USD (new balance: 1002.89 USD)
buying 1.00 CVS (new balance: 939.36 USD)
buying 5.00 CVS (new balance: 621.51 USD)
buying 5.00 CVS (new balance: 301.46 USD)
buying 1.00 UNH (new balance: 50.60 USD)
buying 45.00 NIO (new balance: -253.07 USD)
selling 1.00 UNH (new balance: 6.60 USD)
selling 1.00 NTDOY (new balance: 44.88 USD)
selling 1.00 SQ (new balance: 116.30 USD)
Checking ../data/CRSP.csv
Saving 

selling 10.00 SONO (new balance: 88.90 USD)
Checking ../data/MOMO.csv
Saving ../data/MOMO.csv
Checking ../data/MOMO.csv
Saving ../data/MOMO.csv
buying 28.00 MOMO (new balance: -1044.00 USD)
selling 6.00 BABA (new balance: 30.90 USD)
buying 2.00 HUYA (new balance: -20.01 USD)
buying 2.00 HUYA (new balance: -70.75 USD)
selling 2.00 TCEHY (new balance: 25.41 USD)
dividend ZTO 7.59 USD (new balance: 32.98 USD)
buying 5.00 HUYA (new balance: -88.49 USD)
selling 1.00 OKTA (new balance: 3.61 USD)
buying 16.00 NIO (new balance: -76.38 USD)
buying 4.00 HUYA (new balance: -168.33 USD)
selling 7.00 SONO (new balance: -86.57 USD)
selling 1.00 FTNT (new balance: 3.03 USD)
buying 1.00 MOMO (new balance: -33.84 USD)
selling 3.00 ZTO (new balance: 23.55 USD)
buying 3.00 HUYA (new balance: -44.89 USD)
buying 21.00 NIO (new balance: -139.81 USD)
selling 2.00 CRSP (new balance: -61.33 USD)
selling 1.00 AYX (new balance: 23.44 USD)
buying 2.00 CRSP (new balance: -49.57 USD)
buying 3.00 HUYA (new balance: 

buying 3.00 ZTO (new balance: -2226.79 USD)
selling 1.00 ZEN (new balance: -2133.99 USD)
selling 2.00 HUYA (new balance: -2087.27 USD)
selling 11.00 AYX (new balance: -954.18 USD)
selling 8.00 BYND (new balance: 255.95 USD)
buying 1.00 ZEN (new balance: 168.30 USD)
selling 4.00 SONO (new balance: 213.07 USD)
selling 2.00 ZTO (new balance: 249.69 USD)
buying 1.00 TCEHY (new balance: 206.93 USD)
buying 1.00 JD (new balance: 179.86 USD)
buying 1.00 HUYA (new balance: 157.22 USD)
selling 1.00 SONO (new balance: 168.79 USD)
selling 1.00 SONO (new balance: 180.13 USD)
Checking ../data/JMIA.csv
Saving ../data/JMIA.csv
Checking ../data/JMIA.csv
Saving ../data/JMIA.csv
buying 1.00 JMIA (new balance: 156.72 USD)
buying 3.00 JMIA (new balance: 85.07 USD)
Checking ../data/ZM.csv
Saving ../data/ZM.csv
Checking ../data/ZM.csv
Saving ../data/ZM.csv
buying 11.00 ZM (new balance: -1009.22 USD)
buying 42.00 JMIA (new balance: -2052.92 USD)
selling 4.00 SNAP (new balance: -1995.96 USD)
selling 58.00 ZTO 

buying 1.00 HUYA (new balance: 18.64 USD)
buying 2.00 JMIA (new balance: -26.83 USD)
buying 1.00 WORK (new balance: -60.61 USD)
buying 2.00 HUYA (new balance: -107.30 USD)
buying 2.00 MOMO (new balance: -173.62 USD)
buying 2.00 SNAP (new balance: -203.58 USD)
selling 2.00 OSTK (new balance: -165.83 USD)
selling 1.00 FTNT (new balance: -79.42 USD)
selling 2.00 HUYA (new balance: -29.90 USD)
selling 2.00 OSTK (new balance: 6.90 USD)
buying 8.00 SONO (new balance: -81.68 USD)
buying 1.00 WORK (new balance: -113.85 USD)
buying 1.00 SNAP (new balance: -128.36 USD)
buying 1.00 SNAP (new balance: -142.96 USD)
buying 1.00 JMIA (new balance: -165.18 USD)
buying 3.00 HUYA (new balance: -231.61 USD)
selling 1.00 OKTA (new balance: -92.12 USD)
selling 2.00 OSTK (new balance: -52.76 USD)
selling 1.00 LYFT (new balance: 12.84 USD)
buying 1.00 EVER (new balance: -1.71 USD)
buying 1.00 SONO (new balance: -12.63 USD)
buying 16.00 SONO (new balance: -187.39 USD)
buying 10.00 SONO (new balance: -297.21 U

buying 1.00 JPM (new balance: -102.79 USD)
buying 1.00 WORK (new balance: -133.70 USD)
Checking ../data/LULU.csv
Saving ../data/LULU.csv
Checking ../data/LULU.csv
Saving ../data/LULU.csv
buying 5.00 LULU (new balance: -1035.72 USD)
buying 1.00 SONO (new balance: -1045.55 USD)
buying 1.00 SNAP (new balance: -1061.31 USD)
buying 8.00 JMIA (new balance: -1163.71 USD)
selling 3.00 JMIA (new balance: -1123.36 USD)
selling 1.00 LYFT (new balance: -1062.61 USD)
selling 45.00 OSTK (new balance: -94.21 USD)
selling 1.00 OKTA (new balance: 36.38 USD)
buying 1.00 BYND (new balance: -126.43 USD)
buying 1.00 JPM (new balance: -235.81 USD)
buying 1.00 EVER (new balance: -255.10 USD)
buying 1.00 JPM (new balance: -364.41 USD)
buying 1.00 JPM (new balance: -473.56 USD)
selling 1.00 SONO (new balance: -463.22 USD)
selling 1.00 CRSP (new balance: -413.73 USD)
selling 1.00 LYFT (new balance: -350.95 USD)
selling 1.00 ZM (new balance: -255.86 USD)
selling 1.00 MOMO (new balance: -224.78 USD)
selling 1.00 

In [ ]:
rh.overview()

In [ ]:
rh.overview_archive()

In [ ]:
# parse alternative (SP500) portfolio first
rh_alt = import_portfolio(path="../data/portfolio_rh_alt.csv", name="RobinHood")

In [ ]:
rh_alt.overview()

In [ ]:
rh.payments.sum()

In [ ]:
rh_alt.payments.sum()

In [ ]:
rh.positions()

In [ ]:
rh.get_timeseries()

In [ ]:
rh.data.plot(y=[column for column in rh.data.columns if column not in ['Total', 'Total_sh1', 'Total_sh1_ret', 'Total_sh1_sqerr']], 
               figsize=(12,12)
            )
#plt.ylim((0,2000))
#plt.yscale('symlog')


In [ ]:
plot_composition(portfolio=rh)

In [ ]:
rh.data_growth.plot(y="Growth")

In [ ]:
rh_alt.get_timeseries()

In [ ]:
plt.plot(rh.data_growth.Growth[:], label='MyPortfolio')
plt.plot(rh_alt.data_growth.Growth[:], label='SP500')
plt.legend()

In [ ]:
plt.plot(rh.data_growth.Growth-rh_alt.data_growth.Growth)
plt.hlines(0, xmin=min(rh.data_growth.index), xmax=max(rh.data_growth.index), linestyle='-', linewidth=0.5)

In [ ]:
rh_alt.get_performance()

In [ ]:
rh.get_performance()

In [ ]:
# Volatility of portfolio
df_volatility = standard_deviation_column(rh.data, column='Total', window=1, shift=1)

In [ ]:
rh.get_benchmark()

In [ ]:
rh.get_returns()

In [ ]:
total = et.TotalPortfolioValue(name="Total", data=rh.data) # to-do: turn this into portfolio object

In [ ]:
df_volatility = standard_deviation_column(total.data, column='Total', window=1, shift=1)

In [ ]:
total.get_returns()

In [ ]:
total.get_benchmark()

In [ ]:
total.benchmark.get_returns()

In [ ]:
rsq(total, total.benchmark, col1='Return', col2='Return')

In [ ]:
beta(sec1=total, sec2=total.benchmark, col1='Return', col2='Return')

In [ ]:
alpha(sec1=total, sec2=total.benchmark, col1='Return', col2='Return', risk_free_rate=.004484)

In [ ]:
# show correlation of positions
plot_cross_correlation_matrix(portfolio=rh)

## Put in stop loss orders for Robinhood stocks

In [12]:
from etfs.trader.orders import put_in_stop_loss_orders_all

In [13]:
put_in_stop_loss_orders_all(access_token=access_token, 
                            username=username, 
                            password=password, 
                            portfolio=rh, 
                            column="Close", 
                            alpha=.5, 
                            sigmas=2)

Found Additional pages.
Loading page 2 ...
Loading page 3 ...
Loading page 4 ...
Loading page 5 ...
Loading page 6 ...
Loading page 7 ...
Loading page 8 ...
Loading page 9 ...
Loading page 10 ...
Loading page 11 ...
Loading page 12 ...
Loading page 13 ...
All Orders Cancelled
CRSP 21.0 46.63 47.675180696770255 45.47 -2.205180696770256 -2.4876688826935527
SNAP 63.0 15.43 15.826973048073263 15.26 -0.5669730480732635 -1.1017498379779647
JMIA 79.0 12.98 13.05712904216767 11.66 -1.3971290421676699 -10.16949152542373
ZM 11.0 91.02 92.41998933179266 90.4 -2.019989331792658 -0.6811689738518901
AYX 7.0 138.56 139.68514603100937 134.28 -5.405146031009366 -3.0889145496535804
WORK 33.0 29.7501 30.338172218991307 29.68 -0.6581722189913073 -0.23562946006904195
SHOP 3.0 382.0 383.1369026312318 368.48 -14.656902631231787 -3.5392670157068014
SONO 70.0 13.8 13.46702065337086 12.74 -0.7270206533708592 -7.681159420289859
EVER 45.0 22.4 22.787656718686204 21.21 -1.5776567186862032 -5.31249999999999
BYND 7.

## Vanguard

In [ ]:
vg = import_portfolio_vanguard(path="../data/portfolio_vanguard*.csv", name="Vanguard")

In [ ]:
vg.overview()     

In [ ]:
vg.overview_archive()

In [ ]:
vg.positions()

In [ ]:
vg.get_timeseries()

In [ ]:
ax = vg.data.plot(y=vg.data.columns,
                  figsize=(8,6)
                 )

In [ ]:
plot_composition(portfolio=vg)

In [ ]:
vg.data_growth.plot(y="Growth")

In [ ]:
vg.get_performance()

In [ ]:
vg.get_benchmark()

In [ ]:
# Volatility of portfolio
df_volatility = standard_deviation_column(vg.data, column='Total', window=1, shift=1)

In [ ]:
df_volatility = standard_deviation_column(vg.benchmark.data, column='Close', window=1, shift=1)

In [ ]:
total = et.TotalPortfolioValue(name="Total", data=vg.data) # to-do: turn this into portfolio object

In [ ]:
total.get_benchmark()

In [ ]:
total.get_returns()

In [ ]:
rsq(total, total.benchmark, col1='Return', col2='Return')

In [ ]:
beta(sec1=total, sec2=total.benchmark, col1='Return', col2='Return')

In [ ]:
alpha(sec1=total, sec2=total.benchmark, col1='Return', col2='Return', risk_free_rate=.004484)

In [ ]:
# show correlation of positions
plot_cross_correlation_matrix(portfolio=vg)

## Calculate stop loss prices for Vanguard stocks

In [ ]:
from etfs.trader.orders import get_stop_loss_price
from etfs.stats.basics import ewm_column

In [ ]:
for security in vg.securities:
    last_price = vg.securities[security].get_last_price()
    stop_loss_price = get_stop_loss_price(security=vg.securities[security], column="Close", alpha=.5, sigmas=3)
    print(vg.securities[security].ticker, 
          vg.securities[security].last_price_ewm, 
          stop_loss_price,
          100*(last_price-stop_loss_price)/last_price, 
         )

    df, _ = ewm_column(vg.securities[security].data, column='Close', alpha=0.5, func='mean')
    df, _ = ewm_column(df, column='Close', alpha=0.5, func='std')

    plt.figure(figsize=(20,8))
    plt.plot(df["Close_ewm0.5_mean"])
    plt.plot(df["Close_ewm0.5_mean"]-3*df["Close_ewm0.5_std"])
    plt.show()

## Add up portfolios

In [ ]:
tot = et.Portfolio(name="tot")

In [ ]:
tot = parse_portfolio([rh.transactions, vg.transactions], tot)

In [ ]:
tot.get_timeseries()

In [ ]:
tot.data_growth.plot(y=["Growth"])

In [ ]:
tot.get_performance()

In [ ]:
tot.overview()

In [ ]:
tot.positions()

In [ ]:
tot.overview_df['Growth'] = tot.overview_df['Return']/tot.overview_df['CurrentValue']*100

In [ ]:
ax = tot.overview_df.sort_values(by='Growth', ascending=True).plot(x='Description', 
                                                                   y='Growth', 
                                                                   kind='barh', 
                                                                   legend=False, 
                                                                   title='Growth',
                                                                   figsize=(10,8)
                                                                  )

In [ ]:
ax = tot.positions_df.reset_index().sort_values(by='PercentGrowth', ascending=True).plot(x='Ticker', 
                                                                                         y='PercentGrowth', 
                                                                                         kind='barh', 
                                                                                         legend=False, 
                                                                                         title='PercentGrowth',
                                                                                         figsize=(10,10)
                                                                                        )

In [ ]:
tot.overview_df['PriceChange'] = 100-tot.overview_df['AvgPriceFiFo']/tot.overview_df['LastPrice']*100

In [ ]:
ax = tot.overview_df.sort_values(by='PriceChange', ascending=True).plot(x='Description', 
                                                                        y='PriceChange', 
                                                                        kind='barh', 
                                                                        legend=False, 
                                                                        title='Price Change',
                                                                        figsize=(10,8)
                                                                       )

In [ ]:
tot.overview_archive()

In [ ]:
ax = tot.overview_archive_df.sort_values(by='Return', ascending=True).plot(x='Description', 
                                                                   y='Return', 
                                                                   kind='barh', 
                                                                   legend=False, 
                                                                   title='Return',
                                                                   figsize=(10,8)
                                                                  )

In [ ]:
ax = pd.concat([tot.overview_df, tot.overview_archive_df], axis=0, ignore_index=True, sort=False).sort_values(by='Return', ascending=True).plot(x='Description', 
                                                                   y='Return', 
                                                                   kind='barh', 
                                                                   legend=False, 
                                                                   title='Return',
                                                                   figsize=(10,10)
                                                                  )

In [ ]:
plot_composition(portfolio=tot)

In [ ]:
total = et.TotalPortfolioValue(name="Total", data=tot.data) # to-do: turn this into portfolio object

In [ ]:
total.get_returns()

In [ ]:
total.get_benchmark()

In [ ]:
rsq(sec1=total, sec2=total.benchmark, col1='Return', col2='Return')

In [ ]:
beta(sec1=total, sec2=total.benchmark, col1='Return', col2='Return')

In [ ]:
alpha(sec1=total, sec2=total.benchmark, col1='Return', col2='Return', risk_free_rate=.004484)

In [ ]:
# show correlation of positions
plot_cross_correlation_matrix(portfolio=tot)

## Performance analysis of individual securities

In [ ]:
from etfs.visualization.analysis import plot_security_performance

In [ ]:
plot_security_performance(portfolio=tot, ticker='OKTA')

In [ ]:
for ticker in tot.tickers:
    plot_security_performance(portfolio=tot, ticker=ticker)